# Function to clean the data

In [7]:
import requests
from datetime import datetime, timedelta
import pandas as pd
import warnings

In [8]:
from ipynb.fs.full.Load_Data import load_data

ModuleNotFoundError: No module named 'ipynb'

In [35]:
df = load_data(lat_start = 51, lat_end = 52, long_start = 9, long_end = 10, start_year = 2018, start_month = 4, start_day = 30, delta_hours = 1, combined = True)
df

,measurement,time,lat,lon,Sensor,PM,sensor_id
0,18.43,2018-04-30T00:00:03Z,51.732,9.032,P1,PM10,9946
1,9.17,2018-04-30T00:00:11Z,51.340,9.429,P1,PM10,4784
2,NaN,2018-04-30T00:00:12Z,51.340,9.429,P1,PM10,4785
3,12.87,2018-04-30T00:00:14Z,51.520,9.954,P1,PM10,11998
4,NaN,2018-04-30T00:00:14Z,51.520,9.954,P1,PM10,11999
...,...,...,...,...,...,...,...
1082,9.00,2018-04-30T00:59:50Z,51.290,9.641,P2,PM2.5,4973
1083,NaN,2018-04-30T00:59:51Z,51.290,9.641,P2,PM2.5,4974
1084,11.17,2018-04-30T00:59:54Z,51.854,9.672,P2,PM2.5,9870
1085,12.00,2018-04-30T00:59:57Z,51.561,9.987,P2,PM2.5,12334


In [54]:
# check for missing values and potentially remove those. Also print how many observations have been removed
def remove_missing(df):
    miss = pd.isna(df["measurement"])
    n_miss = miss.sum()
    print(n_miss,"observations were removed from the data frame")
    return df[miss.values == False]    
df = remove_missing(df)

0 observations were removed from the data frame


In [135]:
# check for outliers and filter those using different methods
def remove_outliers(df,method = "Z-score", z_val = 1.96, crit_val = [0,0], quantile = [0,0.95]):
    measurement = df["measurement"]
    
    if method == "Z-score":
        mean = measurement.mean()
        sd = measurement.std()
        z_score = (measurement.values - mean)/sd
        exclude = np.any([z_score < -z_val, z_score > z_val],axis = 0)
    
    if method == "critical_value":
        if len(crit_val) < 2:
            print("ERROR: please provide a lower and an upper bound")
            return
        if len(crit_val) > 2:
            warnings.warn("Only first two elements of crit_val will be used!")
        exclude = np.any([measurement < crit_val[0], measurement > crit_val[1]],axis = 0)
        

    if method == "quantile":
        if len(quantile) < 2:
            print("ERROR: please provide a lower and an upper quantile")
            return 
        if len(quantile) > 2:
            warnings.warn("Only first two elements of quantile will be used!")
        upper = measurement.quantile(quantile[1])
        lower = measurement.quantile(quantile[0])
        exclude = np.any([measurement  < lower, measurement > upper],axis = 0)
    
    n_excluded = exclude.sum()
    print(n_excluded,"observations were deleted")
    return df[exclude == False]
        